In [1]:
pip install GoogleNews

In [ ]:
from GoogleNews import GoogleNews
import json
import time

keywords = ["pigeonpeas", "tur dal", "arhar dal"]
location = "Uttar Pradesh", "Agra"
period = "1y"  # Last 1 month

googlenews = GoogleNews(lang='en', period=period)

all_results = []

for term in keywords:
    query = f"{term} {location}"
    googlenews.search(query)
    results = googlenews.results()

    print(f"🔍 Found {len(results)} results for: {query}")
    
    for item in results:

        if item['title'] and item['link'].startswith("http"):
            cleaned_item = {
                "title": item['title'],
                "media": item['media'],
                "date": item['date'],
                "desc": item['desc'],
                "link": item['link']
            }
            all_results.append(cleaned_item)

    time.sleep(1)

output_file = 'news_link.json'

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, indent=2, ensure_ascii=False)

print(f"\n✅ Saved {len(all_results)} clean news articles to: {output_file}")


🔍 Found 1 results for: pigeonpeas ('Uttar Pradesh', 'Agra')
🔍 Found 8 results for: tur dal ('Uttar Pradesh', 'Agra')
🔍 Found 16 results for: arhar dal ('Uttar Pradesh', 'Agra')

✅ Saved 22 clean news articles to: news_link.json


In [ ]:
from GoogleNews import GoogleNews
import json
import time


with open("crop_queries.json", "r", encoding="utf-8") as f:
    data = json.load(f)

keywords = data["queries"]          
district = data["district"]
prediction = data["prediction"]

period = "1y" 
googlenews = GoogleNews(lang='en', period=period)

all_results = []

for query in keywords:
    googlenews.clear() 
    googlenews.search(query)
    results = googlenews.results()

    print(f"🔍 Found {len(results)} results for: {query}")

    for item in results:
        if item['title'] and item['link'].startswith("http"):
            cleaned_item = {
                "query": query,
                "title": item['title'],
                "media": item['media'],
                "date": item['date'],
                "desc": item['desc'],
                "link": item['link']
            }
            all_results.append(cleaned_item)

    time.sleep(1)

output_file = f'news.json'

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, indent=2, ensure_ascii=False)

print(f"\n✅ Saved {len(all_results)} clean news articles to: {output_file}")


🔍 Found 3 results for: pigeonpeas Agra Uttar Pradesh
🔍 Found 3 results for: pigeonpeas Agra Uttar Pradesh
🔍 Found 10 results for: अरहर Agra Uttar Pradesh
🔍 Found 10 results for: तुअर Agra Uttar Pradesh
🔍 Found 9 results for: tur dal Agra Uttar Pradesh
🔍 Found 2 results for: arhar Agra Uttar Pradesh

✅ Saved 33 clean news articles to: news.json


In [ ]:
import requests
from bs4 import BeautifulSoup

def search_pib_news(keyword="cotton", max_results=10):
    search_url = f"https://pib.gov.in/AllRelease.aspx?kw={keyword}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(search_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        articles = soup.select("div.content-area ul li a")
        results = []

        for a in articles[:max_results]:
            title = a.get_text(strip=True)
            link = "https://pib.gov.in" + a.get("href", "")
            results.append({"title": title, "url": link})

        return results

    except Exception as e:
        print("❌ PIB search failed:", e)
        return []

if __name__ == "__main__":
    keyword = "Pulses"
    news = search_pib_news(keyword)
    if news:
        for i, item in enumerate(news, 1):
            print(f"{i}. {item['title']}\n   🔗 {item['url']}")
    else:
        print("⚠️ No news found.")


1. President of India to visit Odisha from July 14 to 15
   🔗 https://pib.gov.in/PressReleseDetail.aspx?PRID=2144376
2. President of India unveils the Trophies of Durand Cup Tournament
   🔗 https://pib.gov.in/PressReleseDetail.aspx?PRID=2142073
3. PRESIDENT OF INDIA INAUGURATES/LAYS FOUNDATION STONE FOR VARIOUS PROJECTS OF MAHAYOGI GORAKHNATH UNIVERSITY
   🔗 https://pib.gov.in/PressReleseDetail.aspx?PRID=2141322
4. PRESIDENT OF INDIA INAUGURATES MAHAYOGI GURU GORAKHNATH AYUSH UNIVERSITY
   🔗 https://pib.gov.in/PressReleseDetail.aspx?PRID=2141159
5. Text of Vice-President’s address at the 4th Convocation of the Indian Institute of Information Technology (IIIT) in Kota, Rajasthan (Excerpts)
   🔗 https://pib.gov.in/PressReleseDetail.aspx?PRID=2144271
6. Coaching Centres Have Turned Out To Be Poaching Centres; Have Become Black Holes For Talent In Regimented Silos: Vice-President
   🔗 https://pib.gov.in/PressReleseDetail.aspx?PRID=2144236
7. Industry must be a force for inclusion by promot

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_krishi_jagran_fallback(max_articles=10):
    url = "https://krishijagran.com/"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

    
        anchors = soup.find_all("a", href=True)
        seen = set()
        articles = []

        for a in anchors:
            text = a.get_text(strip=True)
            href = a['href']
            if (
                text
                and len(text) > 20 
                and "/news/" in href
                and href not in seen
            ):
                full_url = "https://krishijagran.com" + href if href.startswith("/") else href
                articles.append({"title": text, "url": full_url})
                seen.add(href)
            if len(articles) >= max_articles:
                break

        return articles

    except Exception as e:
        print("❌ Scraper failed:", e)
        return []

# ▶ Run it
if __name__ == "__main__":
    news = scrape_krishi_jagran_fallback()
    if news:
        for i, item in enumerate(news, 1):
            print(f"{i}. {item['title']}\n   🔗 {item['url']}")
    else:
        print("⚠️ Still no output — likely due to local IP or site blocking scraping.")


1. Centre Pushes for Cotton Revolution in Tamil Nadu to Boost Yields and Cut Imports
   🔗 https://krishijagran.com/news/centre-pushes-for-cotton-revolution-in-tamil-nadu-to-boost-yields-and-cut-imports/
2. Weather Update: Heavy Rainfall Alerts for UP, Rajasthan, MP, Odisha, Gujarat, Assam & Kerala as Monsoon Intensifies
   🔗 https://krishijagran.com/news/weather-update-12-07-2025-heavy-rainfall-alerts-for-up-rajasthan-mp-odisha-gujarat-assam-kerala-as-monsoon-intensifies/
3. PM-KISAN 20th Instalment 2025 Big Update: Rs 2,000 Payment May Arrive on This Date, But Not All Farmers Will Get It
   🔗 https://krishijagran.com/news/pm-kisan-20th-instalment-2025-big-update-rs-2-000-payment-may-arrive-on-this-date-but-not-all-farmers-will-get-it/
4. Haryana Govt Extends Deadline for Horticulture Crop Insurance Scheme; Farmers Can Apply Till July 31
   🔗 https://krishijagran.com/news/haryana-govt-extends-deadline-for-horticulture-crop-insurance-scheme-farmers-can-apply-till-july-31/
5. Centre Appr